In [ ]:
%env OPENAI_API_KEY=keyhere

In [1]:
from autogen_agentchat.agents import AssistantAgent, UserProxyAgent
from autogen_agentchat.conditions import MaxMessageTermination, TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import OpenAIChatCompletionClient

import asyncio
from kagent.tools.docs import search_documentation

model_client = OpenAIChatCompletionClient(
    model="gpt-4o",
)

istio_docs_agent = AssistantAgent(
    "istio_docs_agent",
    description="This agent allows you to get data from the Istio docs database.",
    tools=[search_documentation],
    model_client=model_client,
    system_message="""
    You are a support agent.

    You have access to several tools:
    - 'searchDocumentation' to search in the documentation.

    Execute all the following steps:
    1. Product identification
    - Check if you know what product the question is about (Only supported product is: 'istio')
    - If it's not the case, ask what the product is

    2. Information Gathering
    - Search in the documentation for information related to the question the user has submitted

    3. Answer the question
    - Use all the information you gathered to provide a valuable answer to the user
    - Provide links to the documentation whenever possible`,
    """,
)

user_proxy = UserProxyAgent("user_proxy", input_func=input)

text_mention_termination = TextMentionTermination("TERMINATE")
max_messages_termination = MaxMessageTermination(max_messages=25)
termination = text_mention_termination | max_messages_termination


team = RoundRobinGroupChat(
    [
        istio_docs_agent,
    ],
    termination_condition=termination,
)

# task = "How do I set up Istio ambient in my kubernetes cluster?"
task = "Why should I use istio?"

# Use asyncio.run(...) if you are running this in a script.
await Console(team.run_stream(task=task))

---------- user ----------
Why should I use istio?
---------- istio_docs_agent ----------
[FunctionCall(id='call_DdmshNo2Lj9t90EA0GkU2Y7P', arguments='{"query":"why use istio","product":"istio","version":"","limit":5}', name='search_documentation')]
---------- istio_docs_agent ----------
[FunctionExecutionResult(content='[{"distance": 0.812687337398529, "content": "\\nFind out what Istio can do for you.\\n\\n"}, {"distance": 0.8525964617729187, "content": "\\nEnvoy is not inherently multi-tenant. As a result, we have major security and stability concerns with commingling complex processing rules for L7 traffic from multiple unconstrained tenants in a shared instance. Since Kubernetes, by default can schedule a pod from any namespace onto any node, the node is not an appropriate tenancy boundary. Budgeting and cost attribution are also major issues, as L7 processing costs a lot more than L4.\\n\\nIn ambient mode, we strictly limit our ztunnel proxy to L4 processing - [just like the Linu

TaskResult(messages=[TextMessage(source='user', models_usage=None, content='Why should I use istio?', type='TextMessage'), ToolCallRequestEvent(source='istio_docs_agent', models_usage=RequestUsage(prompt_tokens=433, completion_tokens=30), content=[FunctionCall(id='call_DdmshNo2Lj9t90EA0GkU2Y7P', arguments='{"query":"why use istio","product":"istio","version":"","limit":5}', name='search_documentation')], type='ToolCallRequestEvent'), ToolCallExecutionEvent(source='istio_docs_agent', models_usage=None, content=[FunctionExecutionResult(content='[{"distance": 0.812687337398529, "content": "\\nFind out what Istio can do for you.\\n\\n"}, {"distance": 0.8525964617729187, "content": "\\nEnvoy is not inherently multi-tenant. As a result, we have major security and stability concerns with commingling complex processing rules for L7 traffic from multiple unconstrained tenants in a shared instance. Since Kubernetes, by default can schedule a pod from any namespace onto any node, the node is not 

In [ ]:
print(team.dump_component().model_dump_json(indent=2))